In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from gensim.models import FastText
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Attention, Flatten, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [2]:
df = pd.read_csv("tulu_train.csv")
df_val = pd.read_csv("tulu_dev.csv")
df = df[df["Label"].isin(["Not Tulu", "Positive", "Neutral", "Mixed", "Negative"])]
df_val = df_val[df_val["Label"].isin(["Not Tulu", "Positive", "Neutral", "Mixed", "Negative"])]
print(df["Label"].value_counts())
train_len = len(df)
df = pd.concat([df, df_val]).reset_index(drop=True)

Label
Not Tulu    4400
Positive    3769
Neutral     3175
Mixed       1114
Negative     843
Name: count, dtype: int64


In [3]:
nltk.download('stopwords')
nltk.download('punkt')

english_stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\justa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\justa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
def clean_text(text):
    """
    Removes unwanted characters, URLs, special symbols, and repeated characters.
    """
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    text = re.sub(r"[^a-zA-Z\u0C80-\u0CFF\s]", "", text)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    return text.strip()

def transliterate_to_english(text):
    """
    Transliterates Tulu (Kannada script) to English while preserving English words.
    """
    result = []
    for word in text.split():
        try:
            if re.search(r'[\u0C80-\u0CFF]', word):
                word = transliterate(word, sanscript.KANNADA, sanscript.ITRANS)
        except Exception:
            pass
        result.append(word)
    return " ".join(result)

def preprocess_text(text):
    """
    Cleans, transliterates, tokenizes, and removes stopwords.
    """
    text = clean_text(text)
    text = transliterate_to_english(text)
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word.lower() not in english_stopwords]
    return " ".join(tokens)

In [5]:
label_mapping = {
    "Positive": 0,
    "Not Tulu": 1,
    "Neutral": 2,
    "Mixed": 3,
    "Negative": 4  
}
df['Label'] = df['Label'].map(label_mapping)
df['cleaned_text'] = df['Text'].apply(preprocess_text)

In [6]:
X_train = df.iloc[:train_len, 2]
y_train = df.iloc[:train_len, 1]
X_test = df.iloc[train_len:, 2]
y_test = df.iloc[train_len:, 1]

In [7]:
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 300

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

In [9]:
fasttext_model = FastText(sentences=[text.split() for text in X_train], vector_size=EMBEDDING_DIM, window=5, min_count=1, workers=4, alpha=0.1, min_alpha=0.01)

vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

for word, i in word_index.items():
    if word in fasttext_model.wv:
        embedding_matrix[i] = fasttext_model.wv[word]

In [11]:
input_layer = Input(shape=(MAX_SEQUENCE_LENGTH,))

embedding_layer = Embedding(input_dim=vocab_size, output_dim=EMBEDDING_DIM, 
                             weights=[embedding_matrix], trainable=True)(input_layer)

bilstm_layer = Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01)))(embedding_layer)
bilstm_layer = Dropout(0.5)(bilstm_layer)

attention_layer = Attention()([bilstm_layer, bilstm_layer])

flatten = Flatten()(bilstm_layer)
dense_layer = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(flatten)
dense_layer = Dropout(0.5)(dense_layer)  

output_layer = Dense(5, activation='softmax')(dense_layer)

model = Model(inputs=input_layer, outputs=output_layer)

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 300)       │     6,299,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 100, 256)       │       439,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,276,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,016,569 (38.21 MB)

 Trainable params: 10,016,569 (38.21 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
history = model.fit(X_train_padded, np.array(y_train), epochs=5, batch_size=64)

Epoch 1/5
208/208 ━━━━━━━━━━━━━━━━━━━━ 72s 321ms/step - accuracy: 0.5531 - loss: 4.3794
Epoch 2/5
208/208 ━━━━━━━━━━━━━━━━━━━━ 74s 355ms/step - accuracy: 0.6802 - loss: 1.1158
Epoch 3/5
208/208 ━━━━━━━━━━━━━━━━━━━━ 71s 341ms/step - accuracy: 0.7555 - loss: 0.8801
Epoch 4/5
208/208 ━━━━━━━━━━━━━━━━━━━━ 68s 325ms/step - accuracy: 0.7960 - loss: 0.6997
Epoch 5/5
208/208 ━━━━━━━━━━━━━━━━━━━━ 70s 268ms/step - accuracy: 0.8261 - loss: 0.6162


In [13]:
from sklearn.metrics import classification_report

y_pred_probs = model.predict(X_test_padded)
y_pred = np.argmax(y_pred_probs, axis=1)

print(classification_report(y_test, y_pred))

52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step
              precision    recall  f1-score   support

           0       0.81      0.59      0.68       470
           1       0.78      0.85      0.81       543
           2       0.45      0.79      0.58       368
           3       0.32      0.05      0.08       143
           4       0.40      0.15      0.22       118

    accuracy                           0.64      1642
   macro avg       0.55      0.49      0.48      1642
weighted avg       0.65      0.64      0.62      1642

